# install package

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 5.7 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/minhquan6203/A3N1-at-Face-Analysis-Challenge-2024

Cloning into 'Face-Analysis-Challenge'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 160 (delta 91), reused 127 (delta 58), pack-reused 0
Receiving objects: 100% (160/160), 981.52 KiB | 4.16 MiB/s, done.
Resolving deltas: 100% (91/91), done.


# inference yolo

## prepare data & checkpoint

In [3]:
import os
os.makedirs('checkpoint',exist_ok=True)

In [4]:
#https://drive.google.com/file/d/1U7HYArNxPkycEQgcSQ6G51v0EgDk45NB/view?usp=sharing
!curl -L -o 'public_test-001.zip' 'https://drive.usercontent.google.com/download?id=1U7HYArNxPkycEQgcSQ6G51v0EgDk45NB&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2462M  100 2462M    0     0  63.2M      0  0:00:38  0:00:38 --:--:-- 37.4M


In [5]:
#https://drive.google.com/file/d/1ttBF9wdPK6yqtrl_yiZL-Ly7n0T5wJTF/view?usp=sharing
!curl -L -o 'checkpoint/best_yolo.pt' 'https://drive.usercontent.google.com/download?id=1ttBF9wdPK6yqtrl_yiZL-Ly7n0T5wJTF&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 49.6M  100 49.6M    0     0  7401k      0  0:00:06  0:00:06 --:--:-- 13.5M


In [6]:
!unzip /content/public_test-001.zip

Archive:  /content/public_test-001.zip
   creating: public_test/
  inflating: public_test/49074608.jpg  
  inflating: public_test/63701630.jpg  
  inflating: public_test/50869636.jpg  
  inflating: public_test/81768353.jpg  
  inflating: public_test/66790091.jpg  
  inflating: public_test/41766113.jpg  
  inflating: public_test/50009958.jpg  
  inflating: public_test/image_839.jpg  
  inflating: public_test/76578894.jpg  
  inflating: public_test/image_10251.jpg  
  inflating: public_test/59185528.jpg  
  inflating: public_test/69934550.jpg  
  inflating: public_test/69512122.jpg  
  inflating: public_test/image_5943.jpg  
  inflating: public_test/91893621.jpg  
  inflating: public_test/46253307.jpg  
  inflating: public_test/82900603.jpg  
  inflating: public_test/46883284.jpg  
  inflating: public_test/95861632.jpg  
  inflating: public_test/image_5200.jpg  
  inflating: public_test/image_3671.jpg  
  inflating: public_test/image_8042.jpg  
  inflating: public_test/image_8056.jpg  
 

## inference process

In [7]:
from ultralytics import YOLO
import torch
import json
import os
weight_path='checkpoint/best_yolo.pt'
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=YOLO(weight_path).to(device)

In [8]:
bbox=[]
img_name=[]
score=[]
from PIL import Image
import os
from tqdm import tqdm
import pandas as pd

all_file=os.listdir('./public_test')
for idx,img in tqdm(enumerate(all_file),total=len(all_file), desc='infernce yoyo'):
  img_path=os.path.join('./public_test',img)
  results = model(img_path,verbose=False)
  predictions=json.loads(results[0].tojson())
  for pred in predictions:
      x1 = float(pred['box']['x1'])
      y1 = float(pred['box']['y1'])
      x2 = float(pred['box']['x2'])
      y2 = float(pred['box']['y2'])
      xc = (x1 + x2) / 2
      yc = (y1 + y2) / 2
      w = x2 - x1
      h = y2 - y1
      score.append(pred['confidence'])
      bbox.append([x1,y1,w,h])
      img_name.append(img)


data=pd.DataFrame({'file_name':img_name,'bbox':bbox,'score':score})
data.to_csv('test_yolo.csv')

infernce yoyo:   0%|          | 0/2184 [00:00<?, ?it/s]

WARNING ⚠️ NMS time limit 0.550s exceeded


infernce yoyo: 100%|██████████| 2184/2184 [03:13<00:00, 11.29it/s]


## crop image để phân loại các thuộc tính

In [9]:
from PIL import Image
import os
from tqdm import tqdm

def crop_img(folder_path, df, crop_folder):
    os.makedirs(crop_folder, exist_ok=True)
    img_name_list=[]
    id_list=[]
    bbox_list=[]
    score=[]
    for idx, row in tqdm(df.iterrows(), total=len(df), desc='Processing images'):
        img_path = os.path.join(folder_path, row['file_name'])
        img = Image.open(img_path)
        bbox = row['bbox'].replace('[','').replace(']','').replace(' ','').strip().split(',')
        x1, y1, x2, y2 = float(bbox[0]), float(bbox[1]), float(bbox[2]), float(bbox[3])
        x_min, y_min, x_max, y_max =  x1, y1, x2 + x1, y2 + y1
        cropped_img = img.crop([x_min, y_min, x_max, y_max])
        cropped_img.save(os.path.join(crop_folder, f"test_{idx}.jpg"))

        id_list.append(f"test_{idx}.jpg")
        img_name_list.append(row['file_name'])
        bbox_list.append([x1, y1, x2, y2])
        score.append(row['score'])

    df=pd.DataFrame({'file_name':img_name_list,'crop_name':id_list,'bbox':bbox_list, 'score':score })
    df.to_csv('crop_test_info.csv',index=False)

data=pd.read_csv('test_yolo.csv')
folder_path='public_test'
crop_folder='crop_test_image'
crop_img(folder_path,data,crop_folder)

Processing images: 100%|██████████| 2214/2214 [01:12<00:00, 30.43it/s]


# infernce classify task

## prepare data & checkpoint

In [22]:
#svm linear
#https://drive.google.com/file/d/1vLp2MzJ-_ziSIVirLMUgo2r2mpqBLClS/view?usp=sharing
#svm custom
#https://drive.google.com/file/d/1AeHTS0QkJFNpJOtAhEtliv1YFIRS9L7F/view?usp=sharing
#trans
#https://drive.google.com/file/d/1gA4r3MBvjmr9JXG7R3ACkwVWKeFKQaVS/view?usp=sharing
!curl -L -o 'checkpoint/best_model.pth' 'https://drive.usercontent.google.com/download?id=1vLp2MzJ-_ziSIVirLMUgo2r2mpqBLClS&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  331M  100  331M    0     0   122M      0  0:00:02  0:00:02 --:--:--  122M


## lấy tập train để model có thể tạo answer space

In [11]:
import os
os.makedirs('data',exist_ok=True)

In [12]:
#https://drive.google.com/file/d/1XVgtLrsgxBSNStePfwYIkjoUajl1RzX3/view?usp=sharing
!curl -L -o 'data/train.csv' 'https://drive.usercontent.google.com/download?id=1XVgtLrsgxBSNStePfwYIkjoUajl1RzX3&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  861k  100  861k    0     0   884k      0 --:--:-- --:--:-- --:--:--  883k


## inference process

In [23]:
!python /content/Face-Analysis-Challenge/main.py --config /content/Face-Analysis-Challenge/config/svm_config_test.yaml

2024-01-19 17:24:49.381278: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 17:24:49.381338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 17:24:49.383269: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Now evaluate on test data...
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Loading the best model...
Obtaining predictions...
Reading testing data...
100

# make submission

In [24]:
#https://drive.google.com/file/d/19y5hg5po6btLBoq6XHE-WFQmSXabcp8n/view?usp=sharing
!curl -L -o 'public_test_and_submission_guidelin.zip' 'https://drive.usercontent.google.com/download?id=19y5hg5po6btLBoq6XHE-WFQmSXabcp8n&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  858k  100  858k    0     0   670k      0  0:00:01  0:00:01 --:--:--  670k


In [25]:
!unzip public_test_and_submission_guidelin.zip -d submission_guideline

Archive:  public_test_and_submission_guidelin.zip
replace submission_guideline/__MACOSX/._public_test_and_submission_guidelines? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: submission_guideline/__MACOSX/._public_test_and_submission_guidelines  
  inflating: submission_guideline/public_test_and_submission_guidelines/answer.csv  
  inflating: submission_guideline/public_test_and_submission_guidelines/.DS_Store  
  inflating: submission_guideline/__MACOSX/public_test_and_submission_guidelines/._.DS_Store  
  inflating: submission_guideline/__MACOSX/public_test_and_submission_guidelines/._.ipynb_checkpoints  
  inflating: submission_guideline/public_test_and_submission_guidelines/file_name_to_image_id.json  
  inflating: submission_guideline/public_test_and_submission_guidelines/sample_submission.zip  
  inflating: submission_guideline/public_test_and_submission_guidelines/README.pdf  
  inflating: submission_guideline/__MACOSX/public_test_and_submission_guidelines/._README.pdf  


In [26]:
yolo_pred=pd.read_csv('crop_test_info.csv')
multi_task_pred=pd.read_csv('submission.csv')

In [27]:
import json
file_name_to_id=json.load(open('submission_guideline/public_test_and_submission_guidelines/file_name_to_image_id.json','r'))
file_name_to_id.get('68208541.jpg')

940

In [28]:
merged_df = pd.merge(yolo_pred, multi_task_pred, on='crop_name', how='inner')
for i in range(len(merged_df)):
  merged_df['crop_name'][i]=file_name_to_id.get(merged_df['file_name'][i])
merged_df=merged_df.rename(columns={'crop_name':'image_id'})
print(merged_df.shape)
merged_df.head()

<ipython-input-28-a5f7c8e1eab4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['crop_name'][i]=file_name_to_id.get(merged_df['file_name'][i])


(2214, 10)


,file_name,image_id,bbox,score,age,race,masked,skintone,emotion,gender
0,39818460.jpg,273,"[1011.0958251953125, 344.5299072265625, 203.73...",0.915731,20-30s,Mongoloid,unmasked,light,Neutral,Female
1,71088995.jpg,1058,"[1182.19677734375, 412.3304443359375, 149.0874...",0.925076,40-50s,Caucasian,unmasked,light,Neutral,Male
2,image_5848.jpg,2008,"[268.1121520996094, 162.58389282226562, 544.78...",0.942331,Kid,Mongoloid,unmasked,light,Happiness,Female
3,80654118.jpg,1366,"[912.1513671875, 218.23606872558594, 332.07580...",0.911241,40-50s,Mongoloid,masked,mid-light,Neutral,Female
4,7790659.jpg,1290,"[867.44287109375, 334.613037109375, 236.484497...",0.930138,Teenager,Mongoloid,unmasked,light,Happiness,Female


In [29]:
merged_df.drop('score',axis=1).to_csv('answer.csv',index=False)

In [30]:
!zip -r answer.zip answer.csv

  adding: answer.csv (deflated 71%)
